# Singapore Population Forecasting


In [1]:
import pandas as pd
import numpy as np
import requests
import os
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from statsmodels.tsa.arima.model import ARIMA
from scipy.optimize import curve_fit
import warnings
warnings.filterwarnings('ignore')

excel_path = 'Population-1950-2022.xlsx'
if os.path.exists(excel_path):
    df_raw = pd.read_excel(excel_path, skiprows=9)
    df_raw.set_index(df_raw.columns[0], inplace=True)
    try:
        sg_row = df_raw.loc['Total Population (Number)']
    except KeyError:
        mask = df_raw.index.str.contains('Singapore', case=False)
        sg_row = df_raw[mask].iloc[0]
    years = sg_row.index.astype(int)
    population = pd.to_numeric(sg_row.values, errors='coerce')
    df = pd.DataFrame({'year': years, 'population': population}).dropna().sort_values('year').reset_index(drop=True)
else:
    url = 'https://api.worldbank.org/v2/country/SGP/indicator/SP.POP.TOTL?format=json&per_page=1000'
    data = requests.get(url).json()[1]
    df = pd.DataFrame([{'year': int(item['date']), 'population': item['value']} for item in data if item['value'] is not None])
    df = df.sort_values('year').reset_index(drop=True)

df['growth_rate'] = df['population'].pct_change() * 100

train_df = df[df['year'] <= 2019].reset_index(drop=True)
test_df  = df[df['year'] > 2019].reset_index(drop=True)
X_train, y_train = train_df[['year']], train_df['population']
X_test,  y_test  = test_df[['year']],  test_df['population']

lin_reg = LinearRegression().fit(X_train, y_train)
lin_pred = lin_reg.predict(X_test)

arima_model = ARIMA(train_df['population'], order=(2, 2, 1))
arima_res   = arima_model.fit()
arima_pred  = np.array(arima_res.forecast(steps=len(test_df)))

param_grid = {'max_depth': [2,3,4,5,6,7,8,None], 'min_samples_split':[2,5,10], 'min_samples_leaf':[1,2,5]}
dt_search = GridSearchCV(DecisionTreeRegressor(random_state=42), param_grid, cv=5)
dt_search.fit(X_train, y_train)
best_dt = dt_search.best_estimator_
dt_pred = best_dt.predict(X_test)

x_scaler = StandardScaler().fit(X_train)
X_train_scaled = x_scaler.transform(X_train)
X_test_scaled  = x_scaler.transform(X_test)
y_scaler = StandardScaler().fit(y_train.values.reshape(-1,1))
y_train_scaled = y_scaler.transform(y_train.values.reshape(-1,1)).ravel()
mlp = MLPRegressor(hidden_layer_sizes=(20,10), max_iter=5000, random_state=42)
mlp.fit(X_train_scaled, y_train_scaled)
mlp_pred_scaled = mlp.predict(X_test_scaled)
mlp_pred = y_scaler.inverse_transform(mlp_pred_scaled.reshape(-1,1)).ravel()

def logistic(x, K, r, x0):
    return K / (1 + np.exp(-r * (x - x0)))
params, _ = curve_fit(logistic, X_train['year'].values, y_train, p0=[max(y_train)*1.5, 0.03, 1990], maxfev=10000)
log_pred = logistic(X_test['year'].values, *params)

def evaluate(actual, predicted):
    mae = mean_absolute_error(actual, predicted)
    mse = mean_squared_error(actual, predicted)
    rmse = mse ** 0.5
    r2 = r2_score(actual, predicted)
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2}

metrics = {
    'Linear': evaluate(y_test, lin_pred),
    'ARIMA': evaluate(y_test, arima_pred),
    'Decision Tree': evaluate(y_test, dt_pred),
    'Neural Net': evaluate(y_test, mlp_pred),
    'Logistic': evaluate(y_test, log_pred)
}

future_years = np.arange(2025, 2030).reshape(-1, 1)
lin_future   = lin_reg.predict(future_years)
arima_future = np.array(arima_res.forecast(steps=len(future_years)))
dt_future    = best_dt.predict(future_years)
mlp_future   = y_scaler.inverse_transform(mlp.predict(x_scaler.transform(future_years)).reshape(-1,1)).ravel()
log_future   = logistic(future_years.flatten(), *params)
future_df    = pd.DataFrame({
        'year': future_years.flatten(),
        'Linear_Pred': lin_future,
        'ARIMA_Pred': arima_future,
        'DecisionTree_Pred': dt_future,
        'NeuralNet_Pred': mlp_future,
        'Logistic_Pred': log_future
    })

specific_years = np.array([2023, 2030, 2050]).reshape(-1, 1)
lin_specific = lin_reg.predict(specific_years)
max_year = specific_years.max()
last_train_year = train_df['year'].iloc[-1]
steps_needed = int(max_year - last_train_year)
arima_full = np.array(arima_res.forecast(steps=steps_needed))
arima_positions = [int(year - last_train_year - 1) for year in specific_years.flatten()]
arima_specific = arima_full[arima_positions]
dt_specific    = best_dt.predict(specific_years)
mlp_specific   = y_scaler.inverse_transform(mlp.predict(x_scaler.transform(specific_years)).reshape(-1,1)).ravel()
log_specific   = logistic(specific_years.flatten(), *params)
specific_df = pd.DataFrame({
        'year': specific_years.flatten(),
        'Linear_Pred': lin_specific,
        'ARIMA_Pred': arima_specific,
        'DecisionTree_Pred': dt_specific,
        'NeuralNet_Pred': mlp_specific,
        'Logistic_Pred': log_specific
    })

print('Model Performance (test set):')
for name, vals in metrics.items():
    print(f"{name}: MAE={vals['MAE']:.0f}, RMSE={vals['RMSE']:.0f}, R2={vals['R2']:.3f}")
print()
print('Future predictions (2025–2029):')
print(future_df)
print()
print('Predictions for specific years (2023, 2030, 2050):')
print(specific_df)

def ensure_dir(d):
    if not os.path.exists(d):
        os.makedirs(d)
plot_dir = 'plots'
ensure_dir(plot_dir)

# Plot 1
plt.figure(figsize=(10,5.6))
plt.plot(df['year'], df['population'], 'o-', label='Historical Population')
plt.plot(df['year'], lin_reg.predict(df[['year']]), 'r--', label='Linear Fit')
plt.plot(df['year'], logistic(df['year'].values, *params), 'g--', label='Logistic Fit')
plt.plot(future_years, lin_future, 'r^', label='Linear Forecast')
plt.plot(future_years, log_future, 'g^', label='Logistic Forecast')
plt.title('Singapore Population Trend and Forecast')
plt.xlabel('Year')
plt.ylabel('Population')
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(plot_dir, 'population_trend_forecast.png'))
plt.close()

# Plot 2
plt.figure(figsize=(10,5.6))
plt.plot(test_df['year'], y_test, 'ko-', label='Actual')
plt.plot(test_df['year'], lin_pred, 'r^-', label='Linear')
plt.plot(test_df['year'], arima_pred, 'gs-', label='ARIMA')
plt.plot(test_df['year'], dt_pred, 'b*-', label='Decision Tree')
plt.plot(test_df['year'], mlp_pred, 'cD-', label='Neural Net')
plt.title('Model Comparison on Test Set (2020–2024)')
plt.xlabel('Year')
plt.ylabel('Population')
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(plot_dir, 'model_comparison_test.png'))
plt.close()

# Plot 3
plt.figure(figsize=(10,5.6))
plt.bar(df['year'], df['growth_rate'], color='skyblue')
plt.title('Year-on-Year Population Growth Rate (%)')
plt.xlabel('Year')
plt.ylabel('Growth Rate (%)')
plt.axhline(0, color='black', linewidth=0.5)
plt.tight_layout()
plt.savefig(os.path.join(plot_dir, 'growth_rate.png'))
plt.close()


Model Performance (test set):
Linear: MAE=120704, RMSE=149660, R2=-1.241
ARIMA: MAE=250820, RMSE=280227, R2=-6.855
Decision Tree: MAE=111437, RMSE=149717, R2=-1.242
Neural Net: MAE=159358, RMSE=198694, R2=-2.949
Logistic: MAE=592751, RMSE=612919, R2=-36.579

Future predictions (2025–2029):
   year   Linear_Pred    ARIMA_Pred  DecisionTree_Pred  NeuralNet_Pred  \
0  2025  5.775409e+06  5.772075e+06          5703569.0    6.032613e+06   
1  2026  5.842473e+06  5.842489e+06          5703569.0    6.104634e+06   
2  2027  5.909537e+06  5.914291e+06          5703569.0    6.174894e+06   
3  2028  5.976602e+06  5.987117e+06          5703569.0    6.243242e+06   
4  2029  6.043666e+06  6.060702e+06          5703569.0    6.308638e+06   

   Logistic_Pred  
0   6.697208e+06  
1   6.829965e+06  
2   6.964521e+06  
3   7.100865e+06  
4   7.238989e+06  

Predictions for specific years (2023, 2030, 2050):
   year   Linear_Pred    ARIMA_Pred  DecisionTree_Pred  NeuralNet_Pred  \
0  2023  5.641280e+06  5